In [1]:
from reg_utils import *
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [5]:
dim = (192, 240)
#dim = (384, 480)
#dim = (576, 720)
#dim = (768, 960)
#dim = (960, 1200)
coord = True
rgb = True
trainsize = 0.80
validationsize = 0.10
flip = True
noise = 2
X_train, Y_train, X_val, Y_val, X_test, Y_test = load_data(dim, rgb, coord, flip, noise, trainsize, validationsize)

Data set does not exist. Constructing data set.
Importing data...
Import complete.
Dividing and augmenting data...
Dividing data into train and test set.
Adding flipped images to trainset.
Adding images with noise to trainset.
Making Gauss1
Adding Gauss1
Making Gauss2
Adding Gauss2
Saving new data set
Data set construction complete


In [8]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

(4200, 240, 192, 3)
(4200, 2)
(42, 240, 192, 3)
(42, 2)
(43, 240, 192, 3)
(43, 2)


In [6]:
if rgb:
    channels = 3
else:
    channels = 1
    
inshape = (dim[1], dim[0],  channels) 

base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=inshape, pooling=None) #weights='imagenet'
X = GlobalMaxPooling2D()(base_model.output) 
#X = Dense(256, activation='relu')(X)
#X = Dropout(0.7)(X)


if coord:
    output = Dense(2, activation = 'linear')(X) 
else:
    output = Dense(2, activation = 'sigmoid')(X) #'hard_sigmoid'

    
model = Model(base_model.inputs, output)

In [7]:
EPOCHS = 10
BS = 5

for layer in model.layers:
   layer.trainable = True

aug = ImageDataGenerator()#brightness_range=[0.5,1.0])#, shear_range=0.1)

model.compile(optimizer = 'adam', loss='mean_squared_error')

# train the network
model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS), validation_data=(X_val, Y_val), 
                    steps_per_epoch=len(X_train) // BS,
                    epochs=EPOCHS)

Epoch 1/10
840/840 [==============================] - 3551s 4s/step - loss: 2200.2718 - val_loss: 13415.8695
Epoch 2/10
840/840 [==============================] - 3421s 4s/step - loss: 1959.2875 - val_loss: 1676.3746
Epoch 3/10
618/840 [=====================>........] - ETA: 14:45 - loss: 1918.2594

KeyboardInterrupt: 

In [ ]:
y = model.predict(X_test)
print(y)

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=1)

In [ ]:
from math import sqrt
print("Test MSE = " + str(scores))
print("Test RMSE = " + str(sqrt(scores)))